In [1]:
# Imports.
import pygame
import moderngl
import glm
import numpy as np
from LoadObject import getObjectData

pygame 2.6.0 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize Pygame and OpenGL like in prev. Assignments.
def init_window(width=800, height=600):
    pygame.init()
    pygame.display.set_mode((width, height), pygame.OPENGL | pygame.DOUBLEBUF)
    pygame.display.set_caption("Assignment 5: Colin Kirby")
    
    # Create and Set up Context.
    ctx = moderngl.create_context()
    ctx.enable(moderngl.CULL_FACE)
    ctx.enable(moderngl.DEPTH_TEST)
    
    print("Context initialized with depth test and cull face enabled.")
    
    return ctx

In [3]:
# Initialize Window.
ctx = init_window()

Context initialized with depth test and cull face enabled.


In [4]:
# Dictionary to set key inp to Obj.
my_objects = {
    pygame.K_1: "teapot.obj",
    pygame.K_2: "4_tetrahedron.obj",
    pygame.K_3: "6_hexahedron.obj",
    pygame.K_4: "8_octahedron.obj",
    pygame.K_5: "12_dodecahedron.obj",
    pygame.K_6: "20_icosahedron.obj"
}

In [5]:
# Set Default to Teapot.
current_obj = "teapot.obj"
vertex_data, scene_bound = getObjectData(current_obj, normal=True)

Normal exists


In [6]:
# Global VAO obj and Program.
vao = None
program = None

In [7]:
# Upload the vertex data to the GPU
def set_vert_data():
    global vao, program, vertex_data
    print("Uploading vertex data to GPU...")
    
    try:
        # Ensure vertex_data is a numpy array of float32
        vertex_data = np.array(vertex_data, dtype='f4')
        
        # Create a VBO with the current object's data
        vbo = ctx.buffer(vertex_data.tobytes())
        
        # Compile shader program
        program = ctx.program(
            vertex_shader="""
            #version 330
            in vec3 in_position;
            in vec3 in_normal;
            uniform mat4 model;
            uniform mat4 view;
            uniform mat4 proj;
            out vec3 frag_normal;

            void main() {
                frag_normal = in_normal;
                gl_Position = proj * view * model * vec4(in_position, 1.0);
            }
            """,
            fragment_shader="""
            #version 330
            in vec3 frag_normal;
            out vec4 frag_color;

            void main() {
                vec3 color = normalize(frag_normal) * 0.5 + 0.5;  // Compute color from normal
                frag_color = vec4(color, 1.0);
            }
            """
        )
    
        
        # Define the vertex layout (interleaving position and normal data)
        vao = ctx.vertex_array(
            program,
            [
                (vbo, '3f 3f', 'in_position', 'in_normal')  # 3 floats for position, 3 for normal
            ]
        )
        
    except Exception as e:
        print("Error uploading vertex data:", e)


In [8]:
# Upload the vertex data for the initial object
set_vert_data()

Uploading vertex data to GPU...


In [9]:
# Create projection and initial view matrices
fov = 60.0  # Field of view for perspective matrix
width, height = 800, 600
aspect_ratio = width / height
near_plane = 0.1
far_plane = 100.0
bound_radius = 5.0  # Adjust based on the size of the scene

proj_matrix = glm.perspective(glm.radians(fov), aspect_ratio, near_plane, far_plane)

In [10]:
# Global Vars.
global vao, program, current_obj

In [11]:
# Function for handling different inputs and loading objects.
def handle_keypress(event):
    global current_obj, vertex_data, scene_bound
    if event.key in my_objects:
        current_obj = my_objects[event.key]
        vertex_data, scene_bound = getObjectData(current_obj, normal=True)
        set_vert_data()

In [12]:
# Just translations because the teapot was positioned higher than other objects.
translations = {
    "4_tetrahedron.obj": glm.vec3(0.0, 0.0, 0.0),  
    "6_hexahedron.obj": glm.vec3(0.0, 0.0, 0.0),
    "8_octahedron.obj": glm.vec3(0.0, 0.0, 0.0),
    "12_dodecahedron.obj": glm.vec3(0.0, 0.0, 0.0),
    "20_icosahedron.obj": glm.vec3(0.0, 0.0, 0.0),
    "teapot.obj": glm.vec3(0.0, -1.0, 0.0),
}

In [13]:
# Calculate camera position w/ 45 position
def calculate_45_degree_orbit_camera_position(bound_radius, time_in_seconds):
    # Angle of rotation based on time
    angle_in_radian = time_in_seconds
    
    # Calculate the unit vector v, making a 45-degree angle in the XY plane
    v = glm.vec3(np.cos(glm.radians(45.0)), 1.0, np.sin(glm.radians(45.0)))
    
    # Normalize vector to keep 45 degree scale
    v = glm.normalize(v)
    
    # Camera's eyepoint is at twice the bound radius along the unit vector v
    eyepoint = 2 * bound_radius * v

    # Rotate the eyepoint around the Y-axis based on the time (angle)
    rotation_matrix = glm.rotate(glm.mat4(1.0), angle_in_radian, glm.vec3(0.0, 1.0, 0.0))
    rotated_eye_position = rotation_matrix * glm.vec4(eyepoint, 1.0)

    return glm.vec3(rotated_eye_position)



In [ ]:
# Rendering Loop with Camera and Obj Rotation
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            handle_keypress(event)

    # Set up Model Matrix
    model_matrix = glm.mat4(1.0)
    
    # Using Trans. Dict for Placing Obj.
    translation_vector = translations.get(current_obj, glm.vec3(0.0, 0.0, 0.0))
    model_matrix = glm.translate(model_matrix, translation_vector)

    # Get Time for Camera Rotation
    time_in_seconds = pygame.time.get_ticks() / 1000.0

    # Calculate Camera Position for requirements in the assignment
    eye_position = calculate_45_degree_orbit_camera_position(bound_radius, time_in_seconds)

    # Set up the view matrix
    view_matrix = glm.lookAt(
        eye_position,  
        glm.vec3(0.0, 0.0, 0.0), 
        glm.vec3(0.0, 1.0, 0.0)  
    )

    # Write Matrices to Shader
    if vao is not None and program is not None:
        try:
            # Convert Matrices to NP Arrays
            model_bytes = np.array(model_matrix, dtype='f4').T.flatten().tobytes()
            view_bytes = np.array(view_matrix, dtype='f4').T.flatten().tobytes()
            proj_bytes = np.array(proj_matrix, dtype='f4').T.flatten().tobytes()
            
            # Write to Shader Uniforms
            program['model'].write(model_bytes)
            program['view'].write(view_bytes)
            program['proj'].write(proj_bytes)
            
        except Exception as e:
            pass

    # Clear the screen and render
    try:
        ctx.clear(0.1, 0.1, 0.1)
        if vao is not None:
            vao.render(moderngl.TRIANGLES)
    
    except Exception as e:
        pass  # Handle rendering exceptions silently

    # Swap display buffers to show the rendered image
    pygame.display.flip()

pygame.quit()
